In [829]:
import random

In [830]:
#functions for general operations

In [831]:
#encoding function
def encode(msg):
    string_ascii = ''
    for i in msg:
        string_ascii += str(ord(i) + 100)
    return string_ascii

In [832]:
#decoding function
def decode(msg):
    pack = ''
    i = 0
    decoded_msg = ''
    while i < len(str(msg)):
        pack = int(msg[i:i + 3]) - 100
        decoded_msg += chr(pack)
        i = i + 3
    return decoded_msg

In [833]:
def gcd(a1, b1):
    last_rem, rem = abs(a1), abs(b1)
    x, last_x, y, last_y = 0, 1, 1, 0
    while rem:
        last_rem, (q, rem) = rem, divmod(last_rem, rem)
        x, last_x = last_x - q*x, x
        y, last_y = last_y - q*y, y
    return last_rem, last_x * (-1 if a1 < 0 else 1), last_y * (-1 if b1 < 0 else 1)

In [834]:
def mod_inv(a, m):
    g, x, y = gcd(a, m)
    if g != 1:
        raise ValueError
    return x % m

In [835]:
def output(name, content):
    f = open(name, "a")
    f.write(str(content))
    f.close()

In [836]:
#secp224 setup

#prime
P = 0xfffffffffffffffffffffffffffffffffffffffffffffffeffffe56d  

# elliptic curve: y^2 = x^3 + A * x + B
A = 0
B = 5

# generator
Gx = 0xa1455b334df099df30fc28a169a467e9e47075a90f7e650eb6b7a45c
Gy = 0x7e089fed7fba344282cafbd6f7e319f7c0b0bd59e2ca4bdb556d61a5
G = (Gx, Gy)

# order of G
N = 0x10000000000000000000000000001dce8d2ec6184caf0a971769fb1f7

k = random.getrandbits(224)

In [837]:
#functions for ec curves

In [838]:
def double(x1, y1, p=P, a=A):
    s = ((3 * (x1 ** 2) + a) * mod_inv(2 * y1, p)) % p
    x3 = (s ** 2 - x1 - x1) % p
    y3 = (s * (x1 - x3) - y1) % p
    return x3, y3

In [839]:
def add(x1, y1, x2, y2, p=P, a=A):
    s = 0
    if x1 == x2:
        s = ((3 * (x1 ** 2) + a) * mod_inv(2 * y1, p)) % p
    else:
        s = ((y2 - y1) * mod_inv(x2 - x1, p)) % p
    x3 = (s ** 2 - x1 - x2) % p
    y3 = (s * (x1 - x3) - y1) % p
    return x3, y3

In [840]:
def multiply(mul, gen, p=P, a=A):
    (x3, y3) = (0, 0)
    (x1, y1) = gen
    (x_tmp, y_tmp) = gen
    init = 0
    for i in str(bin(mul)[2:]):
        if (i == '1') and (init == 0):
            init = 1
        elif (i == '1') and (init == 1):
            (x3, y3) = double(x_tmp, y_tmp, p, a)
            (x3, y3) = add(x1, y1, x3, y3, p, a)
            (x_tmp, y_tmp) = (x3, y3)
        else:
            (x3, y3) = double(x_tmp, y_tmp, p, a)
            (x_tmp, y_tmp) = (x3, y3)
    return x3, y3

In [841]:
#generating key for encryption and decryption
private_key = random.getrandbits(224)

In [842]:
def keygen():
    # public key
    return multiply(private_key, G)

In [843]:
# encryption
def encrypt(public_key, txt):
    C1 = ec_multiply(k, G)
    C2 = ec_multiply(k, public_key)[0] + int(txt)
    return C1, C2

In [844]:
# decryption
def decrypt(C1, C2, private_Key):
    return C2 - multiply(private_Key, C1)[0]

In [845]:
#output files and test cases
def main():
    print("Here is the Setup: ")
    #public key
    public_key = keygen()
    print("The Public Key is: ", public_key)
    output("public_key.txt", public_key)

    #Generator
    print("The Generator Point is: ", G)

    #private key
    print("The Private Key is: ", private_key)
    print("\n")
    output("private_key.txt", private_key)

    #test case 1
    with open('test_case_1.txt', 'r') as file:
        test_case_1 = file.read()

    print("Test Case 1:")
    (C1, C2) = encrypt(public_key, encode(test_case_1))
    string = decrypt(C1, C2, private_key)
    decrypted_text = decode(str(string))
    print("The Cipher Text for Test Case 1 : ", C1, C2)
    print("The Decrypted Message: ", decrypted_text)
    print("\n")
    output("decrypted_test_case_1.txt", decrypted_text)
    
    with open('test_case_2.txt', 'r') as file:
        test_case_2 = file.read()

    print("Test Case 2:")
    (C1, C2) = encrypt(public_key, encode(test_case_2))
    string = decrypt(C1, C2, private_key)
    decrypted_text = decode(str(string))
    print("The Cipher Text for Test Case 2 : ", C1, C2)
    print("The Decrypted Message: ",decrypted_text)
    output("decrypted_test_case_2.txt", decrypted_text)

    #creating additional files
    output("c1.txt", C1)
    output("c2.txt", C2)


In [846]:
if __name__ == "__main__":
    main()

Here is the Setup: 
The Public Key is:  (16428242821365106946151494941740555397256339467233621272139904620624, 989628995026326989797369769111949656992692707714858091030015380389)
The Generator Point is:  (16983810465656793445178183341822322175883642221536626637512293983324, 13272896753306862154536785447615077600479862871316829862783613755813)
The Private Key is:  23202481220577210482354000291693773088776892885554781306404576440131


Test Case 1:
The Cipher Text for Test Case 1 :  (10696031291557086741855538900367261947129284539067787205167278353296, 26917573109710817229673303951069899819426876009205702465923730276792) 173132197209132197210132217210200201214203214197200217197216201132215216217200201210236179625478756477839792010388060498136767918221305942483837431889175
The Decrypted Message:  I am an undergraduate student at queen's university


Test Case 2:
The Cipher Text for Test Case 2 :  (10696031291557086741855538900367261947129284539067787205167278353296, 26917573109710817229673